In [63]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

In [64]:
# path to your service account key JSON file
key_file_path = "./smartwaiver-388021-9a8ea3b56852.json"

# Use your JSON key to authenticate your account
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_path, scope)

client = gspread.authorize(creds)


In [65]:
# Open the Google Spreadsheet by its name (make sure you've shared it with the client_email from your service account)
sheet = client.open('smartwaiver_test').sheet1

In [66]:
# put the full file name for the smartwaiver data here
smartwaiver = 'smartwaiver-173467-647a0861807c9.csv'

# load the data from your CSV file
data = pd.read_csv(smartwaiver)

In [67]:
# convert 'Waiver Date' column to datetime
data['Date Completed (UTC)'] = pd.to_datetime(data['Date Completed (UTC)'])

In [68]:
# get the current date
now = datetime.now()

# calculate the date one year ago
one_year_ago = now - timedelta(days=365)

In [69]:
# filter the data to get volunteers who signed a waiver in the past year
filtered_data = data[data['Date Completed (UTC)'] >= one_year_ago]

In [70]:
filtered_data.head(5)

,First,Middle,Last,Date of Birth,Date Completed (UTC),Waiver ID,Status,Check-ins,Gender,Phone,...,Parent/Guardian First Name,Parent/Guardian Middle Name,Parent/Guardian Last Name,Parent/Guardian Phone,Parent/Guardian Relationship,Email,signing_type,template_type,Title of Document,typed_font
0,Patrick,NaN,Hayes,12/11/2007,2023-04-26 14:58:52,fgEHFCkdNYZJJMuTgY8BzU,Completed Online,0,NaN,NaN,...,Amy,NaN,Hayes,NaN,NaN,aabigailh@gmail.com,draw,1,SCMTS Waiver,dancing
1,Mark,NaN,Phillips,NaN,2023-04-26 15:03:37,EFyeg52WYcWnW9wj7VjfZ8,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,mark.d.phillips.2555@gmail.com,draw,1,SCMTS Waiver,dancing
2,Peter,NaN,Nell,NaN,2023-04-26 15:05:28,Mhxdw3q6rrhNVigGQbnH8E,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,peteralbertnell@gmail.com,draw,1,SCMTS Waiver,dancing
3,Raven,NaN,Adams,NaN,2023-04-26 15:06:14,KjmejHvvZEfNoQRJUnbsz6,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ravenadams08@gmail.com,draw,1,SCMTS Waiver,dancing
4,Elise,NaN,Roy,NaN,2023-04-26 15:08:17,P4pDstc74pMuYuhV8orcoB,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,elise.c.roy@gmail.com,draw,1,SCMTS Waiver,dancing


In [71]:
# Get all records from the Google Sheet
records = sheet.get_all_records()

# Create an empty DataFrame to store unmatched names
unmatched_names = pd.DataFrame(columns=['First Name', 'Last Name', 'Email'])

In [72]:

# iterate over the filtered data
for index, row in filtered_data.iterrows():
    
    # find the volunteer in the Google Spreadsheet
    matched_records = [record for record in records if record['Contact Name'].split(", ")[1] == row['First'] and record['Contact Name'].split(", ")[0] == row['Last']]

    if matched_records:
          
        # if there is a match check the box
        record = matched_records[0]
        
        row_number = records.index(record) + 2 # Adds 2 because gspread row index starts at 1 and we have a header row

        # check the box in the column next to the volunteer (change 'B' to your actual column)
        sheet.update_cell(row_number, 4, True) # The number 3 reflects the column number

else:
    # if there's no match, add the name to the DataFrame
        unmatched_names = unmatched_names.append({'First': row['First'], 'Last': row['Last']}, ignore_index=True)

In [73]:
records

[{'VANID': 130639252,
  'Check-in': 'FALSE',
  'Contact Name': 'Butden, Mark',
  'SCMTS Waiver': 'FALSE',
  'Preferred Email': 'markbaywolf@yahoo.com',
  ' Emergency Contact Name': 'Sheilah Cummings',
  'Emergency Contact Phone Number': 8314308696},
 {'VANID': 125408108,
  'Check-in': 'FALSE',
  'Contact Name': 'Chowaniec, Phil',
  'SCMTS Waiver': 'FALSE',
  'Preferred Email': 'phil.chowaniec@gmail.com',
  ' Emergency Contact Name': 'Jennifer Chowaniec',
  'Emergency Contact Phone Number': 4086550568},
 {'VANID': 127542108,
  'Check-in': 'FALSE',
  'Contact Name': 'Cummings, Sheilah',
  'SCMTS Waiver': 'FALSE',
  'Preferred Email': 'shecummings@yahoo.com',
  ' Emergency Contact Name': 'Mark Burden',
  'Emergency Contact Phone Number': 8314308490},
 {'VANID': 128824937,
  'Check-in': 'FALSE',
  'Contact Name': 'Giuliano-Monroy, Kaiya',
  'SCMTS Waiver': 'FALSE',
  'Preferred Email': 'kgiuliano@coastal-watershed.org',
  ' Emergency Contact Name': '',
  'Emergency Contact Phone Number': '

In [74]:
unmatched_names

,First Name,Last Name,Email,First,Last
0,NaN,NaN,NaN,Tiare,Slagle
